# Import fNIRS Data into HyPyP

**Authors**: Ghazaleh Ranjbaran, Caitriona Douglas, Guillaume Dumas  
**Date**: 2022-05-21

## Overview

This notebook demonstrates how to import fNIRS data into HyPyP. The steps include:

1. **Loading required libraries** for interactive plotting, numerical processing, and EEG/fNIRS data handling.
2. **Loading fNIRS tools** from HyPyP for reading data, creating montages, and epoching.
3. **Setting the data paths** for SNIRF files (fNIRS format).
4. **Loading the fNIRS data** using MNE functions wrapped by HyPyP.
5. **Creating a compatible montage** using probe information and standard sensor definitions.
6. **Creating Epoch objects** compatible with HyPyP for further analyses.
7. **Plotting the data** to inspect sensor locations and time courses.

For more details on the supported formats and montage creation, please refer to the [MNE fNIRS tutorial](https://mne.tools/stable/auto_tutorials/io/30_reading_fnirs_data.html).

### Loading useful libraries

In [1]:
# Enable interactive plotting in a separate window
%matplotlib qt

# Import necessary libraries for numerical operations and data handling
import numpy as np
import mne
import os

print('Basic libraries loaded.')

Basic libraries loaded.


### Loading fNIRS Tools

HyPyP provides specialized functions for fNIRS processing. Here we import functions for:

- **load_fnirs**: Loading fNIRS data in various formats.
- **make_fnirs_montage**: Creating a sensor montage compatible with MNE.
- **fnirs_epoch**: Converting fNIRS data into MNE Epochs objects.
- **fnirs_montage_ui**: A helper to build montage inputs interactively.

No custom functions are defined here; these are imported from HyPyP.

In [2]:
from hypyp.fnirs_tools import load_fnirs
from hypyp.fnirs_tools import make_fnirs_montage
from hypyp.fnirs_tools import fnirs_epoch

print('fNIRS tools loaded.')

fNIRS tools loaded.


### Step 1: Setting the Path

Specify the file paths for the fNIRS data. In this example, the data is in SNIRF format and does not include an explicit montage file. Data can be found, for instance, at [this OSF link](https://osf.io/75fet/).

In [3]:
# Define file paths for the SNIRF data for two participants
path_1 = "../data/FNIRS/DCARE_02_sub1.snirf"
path_2 = "../data/FNIRS/DCARE_02_sub2.snirf"

print('Data paths set:')
print('Participant 1:', path_1)
print('Participant 2:', path_2)

Data paths set:
Participant 1: ../data/FNIRS/DCARE_02_sub1.snirf
Participant 2: ../data/FNIRS/DCARE_02_sub2.snirf


### Step 2: Loading fNIRS Data

HyPyP utilizes the MNE-Python library to load fNIRS data. Currently, four data types are supported:

- **SNIRF** (.snirf)
- **NIRx** (directory or hdr) – requires NIRStar version 15.0+ or Aurora version 2021+
- **Hitachi** (.csv)
- **BOXY** (.txt)

More information can be found [here](https://mne.tools/stable/auto_tutorials/io/30_reading_fnirs_data.html).

We load the data for both participants using the `load_fnirs` function. Note that the function returns a list of fNIRS data objects.

In [4]:
# Load fNIRS data for two participants
fnirs_data = load_fnirs(path_1, path_2, attr=None, preload=False, verbose=None)
fnirs_participant_1 = fnirs_data[0]
fnirs_participant_2 = fnirs_data[1]

print('fNIRS data loaded for both participants.')

Loading /Users/blackstar/PPSP/02 - Dev/020 - Dev - Hypyp/Sandbox/HyPyP/tutorial/../data/FNIRS/DCARE_02_sub1.snirf
Loading /Users/blackstar/PPSP/02 - Dev/020 - Dev - Hypyp/Sandbox/HyPyP/tutorial/../data/FNIRS/DCARE_02_sub2.snirf
fNIRS data loaded for both participants.


### Step 3: Creating a Compatible Montage

A montage defines the spatial configuration of sensors. MNE-Python offers built-in standard montages for some fNIRS devices (e.g., 'artinis-octamon' or 'artinis-brite23').

There are several approaches to create a montage:

1. **Using a vendor name**: Pass the vendor's name (e.g., `'artinis-octamon'`) via the `mne_standard` argument if compatible.
2. **Using a custom file**: If a custom montage file is available, pass its directory to the `prob_directory` argument and set `create_montage` to `False`.
3. **Interactive montage creation**: Use `fnirs_montage_ui()` to build montage inputs and then call `make_fnirs_montage()` with those parameters.

In this example, we manually specify the montage inputs using a probe information file.

Or we can create the inputs list manually and use it in the same way as above 

In [5]:
# Define source and detector labels
source_labels = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8']
detector_labels = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8']

# Path to the probe information file
prob_mat_file = '../data/FNIRS/MCARE_01_probeInfo.mat'

# 3D coordinates (in mm) of anatomical landmarks
Nz_coord = [12.62, 17.33, 16.74]    # Tip of the nose
RPA = [21.0121020904262, 15.9632489747085, 17.2796094659563]    # Right preauricular
LPA = [4.55522116441745, 14.6744377188919, 18.3544292678269]     # Left preauricular

# Head size in mm
head_size = 0.16

# Create the montage using the provided probe information
location = make_fnirs_montage(source_labels, detector_labels, prob_mat_file,
                              Nz_coord, RPA, LPA, head_size)

print('Compatible montage created.')

Compatible montage created.


### Step 4: Creating Epoch Objects

HyPyP is compatible with MNE Epochs. In this step, we convert the loaded fNIRS data into Epoch objects. The parameters `tmin`, `tmax`, and `baseline` are set according to the specifics of the fNIRS recording. Adjust these values as needed; refer to the [MNE Epochs documentation](https://mne.tools/stable/generated/mne.Epochs.html) for more details.

The function `fnirs_epoch` returns a list of epoch objects – one per participant.

In [6]:
# Create Epoch objects for both participants
fnirs_epochs = fnirs_epoch(fnirs_participant_1, fnirs_participant_2,
                           tmin=-0.1, tmax=1, baseline=(None, 0),
                           preload=True, event_repeated='merge')

fnirs_epo1 = fnirs_epochs[0]
fnirs_epo2 = fnirs_epochs[1]

print('Epoch objects created for both participants.')

Used Annotations descriptions: [np.str_('1'), np.str_('2'), np.str_('3'), np.str_('4'), np.str_('5')]
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
16 matching events found
Setting baseline interval to [-0.128, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 16 events and 10 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('1'), np.str_('2'), np.str_('3'), np.str_('4'), np.str_('5')]
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
16 matching events found
Setting baseline interval to [-0.128, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 16 events and 10 original time points ...
0 bad epochs dropped
Epoch objects created for both participants.


fnirs_epo1 and fnirs_epo2 are now compatible with tools provided by HyPyP (for visualization and statictical analyses check [this](https://github.com/ppsp-team/HyPyP/blob/master/tutorial/getting_started.ipynb) tutorial)

### Step 5: Plotting the Data

Finally, we apply the montage to the epoch objects and plot the sensor configuration and time series. This allows for a visual inspection of the sensor layout and the recorded signals.

In [7]:
# Set the montage for both epoch objects
fnirs_epo1.set_montage(location)
fnirs_epo2.set_montage(location)

print('Montage applied to both participants.')

Montage applied to both participants.


In [8]:
# Plot sensor locations with channel names for participant 1
fnirs_epo1.plot_sensors(show_names=True)

# Plot sensor locations with channel names for participant 2
fnirs_epo2.plot_sensors(show_names=True)

print('Sensor plots displayed.')

Sensor plots displayed.


2025-02-25 14:45:14.925 python[56804:1014910] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-25 14:45:14.925 python[56804:1014910] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [9]:
# Plot the fNIRS data for participant 1
fnirs_epo1.plot()

# Plot the fNIRS data for participant 2
fnirs_epo2.plot()

print('fNIRS data plots displayed.')

Using matplotlib as 2D backend.
fNIRS data plots displayed.
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
